## Image Analysis of Marangoni Cell via YOLOv4 tiny  
### - 지구과학교육과 16학번 한동환(handh21@snu.ac.kr)


1. Install Darknet

  Darknet이라 하는 YOLO 알고리즘의 중추적인 역할을 하는 프로그램이 담긴 파일 다운로드 받기 
2. Making Images Labels

  YOLO 알고리즘에 학습 시키기 위해서는 이미지 내의 객체에 대한 정보가 필요하다.
  
  객체 종류 / 중심 x 위치 / 중심 y 위치 / 너비 / 높이

  이를 labelImg-master라는 프로그램을 통해 쉽게 만들 수 있다.

  자세한 설명은 하단 참조

3. YOLO 알고리즘 설정하기

4. Learning - iteration: 4000~10000

  알고리즘 학습 단계
5. Prediction

  학습한 알고리즘을 통해 이미지를 인식시켜보는 단계





### 1. Install Darknet

Google colab에 YOLO v4 -tiny 버전을  설치하는 것을 기준으로 하였다.

아래의 순서대로 차근차근 설치하면 된다.

YOLO v3, v3-tiny, v4, v5와 같은 다른 버전도 설치 가능

만약 GPU가 있는 서버를 이용한다면 Mobaxterm이나 Putty에서 이용 가능

In [ ]:
%%capture
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
import re
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
#utility function
## Google Colab 환경에서는 cv2.imshow 함수가 잘 작동하지 않아 이를 대체할 함수를 미리 만들어준다.
## Colab에서 실행시키는 것이 아니라 자체 서버에서 실행시키는 경우에는 생략가능하다. 

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
repo_url = 'https://github.com/GotG/yolotinyv3_medmask_demo'
import os
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}

/content
Cloning into 'yolotinyv3_medmask_demo'...
remote: Enumerating objects: 1733, done.
remote: Total 1733 (delta 0), reused 0 (delta 0), pack-reused 1733
Receiving objects: 100% (1733/1733), 208.88 MiB | 34.13 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Checking out files: 100% (1537/1537), done.
/content/yolotinyv3_medmask_demo


이 단계가 설치에서 매우 중요하다. YOLO 알고리즘을 이용해서 인식하고자하는 객체가 있을 것이다. 그것이 어떤 것인지 여기에 입력을 해주어야한다. 입력하는 순서에 따라 class의 number가 결정된다.

가령, 이 알고리즘을 통해 이미지 속의 고양이, 강아지, 캥거루, 곰, 코알라를 구별하고자 한다면 아래에서

labels = ['cat', 'dog', 'kangaroo', 'bear', 'koala'] 라 입력해야한다.

이 경우, cat이 0번 클래스, dog이 1번 클래스, kangaroo가 2번 클래스, bear가 3번 클래스, koala가 4번 클래스가 된다.

In [ ]:
###################################### Setting the Label #############################################

labels_path = '/content/yolotinyv3_medmask_demo/obj.names'

# 이 부분 반드시 수정하기!!! ================================================
labels = ['marangoni cell', 'scale bar', 'information']
# ===========================================================================

with open(labels_path, 'w') as f:

    f.write('\n'.join(labels))

#check that the labels file is correct
!cat /content/yolotinyv3_medmask_demo/obj.names

marangoni cell
scale bar
information

YOLO 알고리즘을 학습시키기 위해서는 obj.data 파일에 크게 5개의 정보를 입력해주어야 한다.

1. 총 몇 종류의 객체들을 인식할 것인지를 나타내는 classes 정보

   (가령, 고양이, 강아지, 캥거루, 곰, 코알라인 경우에는 총 5개 종류의 객체를 인식하므로 classes = 5)

2. 어떤 이미지들을 학습에 사용할 것인지 경로를 나타내는 train.txt파일
   
   (가령, 

  /content/drive/MyDrive/cbe_paper/data_augmt3/coarse_salt_15.jpg
/content/drive/MyDrive/cbe_paper/AgNW_30_001/AgNW_30_001.avi_000007099.jpg
/content/drive/MyDrive/cbe_paper/data_augmt3/log_contrast_15.jpg

  와 같이 경로들이 나열되어 있는 텍스트 파일이 어느 위치에 있는지 입력해야한다.)

  이러한 텍스트 파일을 어떻게 만드는지는 make_sets_txtfile 파일 참조.

3. 어떤 이미지들을 테스트에 사용할 것인지 경로를 나타내는 valid.txt파일

   (2.와 동일)

4. 어떤 객체들을 인식할 것인지 그 이름들을 나열한 obj.names 파일

5. iteration이 진행됨에 따라 weight 파일이 업데이트 될 텐데, 이러한 학습 기록을 어느 폴더에 저장할 것인지를 나타내는 backup 폴더경로



In [ ]:
import re
objdata = '/content/yolotinyv3_medmask_demo/obj.data'
with open(objdata) as f:
    s = f.read()

#the number of classes is equal to the number of labels
num_classes = len(labels)   
s = re.sub('classes= \d*','classes= ' + str(num_classes),s)

with open(objdata, 'w') as f:
  f.write(s)
!cat /content/yolotinyv3_medmask_demo/obj.data

classes= 3
train  = /content/yolotinyv3_medmask_demo/train.txt
valid  = /content/yolotinyv3_medmask_demo/valid.txt
names = /content/yolotinyv3_medmask_demo/obj.names
backup = backup/

In [ ]:
%cd ../yolotinyv3_medmask_demo/

/content/yolotinyv3_medmask_demo


Google Colab에 그냥 저장시키면 런타임이 지나고 기록이 삭제되어버리므로 Google Drive에 저장하기 위해 파일을 옮긴다.

자체 서버를 이용하여 진행하는 경우 이 단계 생략 가능

In [ ]:
!mv /content/yolotinyv3_medmask_demo /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo
!mv /content/darknet /content/drive/MyDrive/cbe_paper/darknet

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
mv: cannot overwrite non-directory '/content/drive/MyDrive/cbe_paper/darknet/darknet' with directory '/content/darknet'


### 2. Making Image Label

  YOLO 알고리즘에 학습 시키기 위해서는 이미지 내의 객체에 대한 정보가 필요하다.
  
  객체 종류 / 중심 x 위치 / 중심 y 위치 / 너비 / 높이

  위와 같은 정보가 담겨 있는 텍스트 파일을 이미지 마다 하나씩 만들어야 한다.

  가령, /desktop/dog_picture/dog1.jpg라는 이미지가 있다면, 그와 동일한 이름을 가진 /desktop/dog_picture/dog1.txt 파일을 만든 뒤에 위의 정보를 이 안에 입력해야하는 것이다.

  이를 labelImg-master라는 프로그램을 이용하면 드레그를 통해 쉽게 만들 수 있다.


  프로그램 다운로드 방법은 https://github.com/tzutalin/labelImg#labelimg 에서 확인할 수 있다.

파일 다운로드 후 커맨드 창에 labelImg.py라는 파일이 위치한 곳으로 경로를 설정한 뒤에 python labelImg.py라 입력하면 프로그램이 실행된다.

### 3. YOLO 알고리즘 설정하기

YOLO 알고리즘과 관련한 여러 설정들은 yolov4-tiny.cfg라는 파일 내에서 조절이 가능하다. 아래를 보면 이 파일이 어떻게 구성되어 있는지 확인 가능하다. 이 파일을 수정함으로써 설정을 변경할 수 있다. mobaxterm이나 putty를 사용하는 경우에는 직접 파일에 들어가서 수정이 가능하다. Google Colab 내에서는 아래와 같은 방법으로 설정을 바꾸는 것이 가장 편리하다.

In [ ]:
!head -n 1000 /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg

가장 먼저 수정해야할 것은 max_batch이다. 이 것은 알고리즘이 총 몇 iteration 동안 학습을 할 것인지를 나타낸다. 가령, 10000 iteration을 학습할 것이면 max_batch를 10000이라 설정해야한다. 일반적으로 calss의 갯수에 2000을 곱한 값까지는 최소한 학습 해주어야 한다고 알려져있다. 그렇다고 너무 숫자를 크게하면 오버피팅이 될 수 있으므로 학습 추이를 보며 적절할 때 학습을 끝내주어야 한다.

또한, 두번째로 수정해야할 것은 num_filters이다. 이 값은 (class의 수 + 5) * 3이 되어야 한다. 그 이유는 YOLO 알고리즘이 객체의 width, height, x, y, confidence로 구성된 총 5가지 요소를 고려하고 각각의 grid cell마다 총 3개의 box를 detect하기 때문이다. 만약, Bounding Box를 기울어지게 설정하여 angle이라는 요소를 추가하게 된다면 width, height, x, y, confidence에 angle이 추가되므로 5가 아닌 6을 더해야할 것이다.

맨 마지막으로는 GPU 사용 여부를 입력해야한다. 당연히 GPU가 있는 경우 아래와 같이 수정하면 된다. 

In [ ]:
import re
# set the number of max_batches - min 2000 per class:
max_batch= 10000
# calculate the 2 steps values:
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch
#subdivisions define the minibatch size: minibatch = batch/subdivisions
#we set subdivisions to 4. if using larger resolutions, may have to increase
#the this number to 8 16 or even 32. default resolution is 416x416
subdivisions = 4


# we also need to adjust the number of classes and a parameter called filter size 
# that are both is inside the model structure

# num_classes = len(labels)
num_filters = (3 + 5) * 3

cfg_file = '/content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg'

with open(cfg_file) as f:
    s = f.read()
# (re.sub('[a-z]*@', 'ABC@', s))
s = re.sub('subdivisions=\d*','subdivisions='+str(subdivisions),s)
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(3),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)
# pad=1\nfilters=\d\d
s = re.sub('CUDNN=0','CUDNN=1',s)
s = re.sub('OPENCV=0','OPENCV=1',s)

with open(cfg_file, 'w') as f:
  s = re.sub('GPU=0','GPU=1',s)
  f.write(s)

###3. Training

이제 알고리즘 학습을 위한 모든 준비가 끝났다. 아래와 같이 입력하면 학습이 가능하다. Iteration이 진행됨에 따라 weight파일이 업데이트 되게 되는데, weight 파일이 용량이 꽤 크기 때문에 매 iteration마다 backup 폴더에 저장하는 것은 비효율적이다. 그래서 기본 설정으로는 10000iteration마다 저장 되게 설정이 되어있다. 

만약 이를 100iteraton이나 1000iteration으로 바꾸고 싶다면 darknet 폴더를 들어가서 src 파일 내의 detector.c 내부의 설정을 바꾸고 !make를 커맨드 창에 입력해서 darknet을 다시 설치해주면 된다. 

In [ ]:
!/content/darknet/darknet detector train /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.conv.29 -dont_show -ext_output -map

###4. Test

학습된 알고리즘이 얼마나 testset을 잘 인식하는지를 map를 기준으로 확인이 가능하다. 몇 iteration일 때의 학습결과를 사용할 것인지 저장된 weight 파일을 지정하면 그 학습 기록을 토대로 알고리즘이 testset을 인식하고 그 결과를 바로 보여준다. 

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_best.weights" -points 0

/bin/bash: /content/darknet/darknet: No such file or directory


In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_1000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_2000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_3000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_4000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_5000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_6000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_7000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_8000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_9000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

In [ ]:
!/content/darknet/darknet detector map /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_10000.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     

### 5. Prediction

전체적인 인식율 이외로, 직접 이미지 상에서 어떻게 인식하는지 보고 싶다면 아래와 같이 weight 파일을 지정하고 어느 이미지를 인식할지 이미지 경로를 입력해주면 결과를 확인할 수 있다. darknet 폴더 안에 들어가면 prediction.jpg로 알고리즘이 이미지 내에서 예측한 결과를 확인할 수 있다.

In [ ]:
!/content/darknet/darknet detector test /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/obj.data  /content/drive/MyDrive/cbe_paper/yolotinyv3_medmask_demo/yolov4-tiny.cfg  "/content/drive/MyDrive/cbe_paper/backup/yolov4-tiny_best.weights" /content/drive/MyDrive/cbe_paper/test_rod_particle_marangoni_bbx/Rod_particle_marangoni_00_00_07_02_218.jpg -ext_output -threshold 50

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	     